In [15]:
import torch
import onnx
# import netron

In [ ]:
print(torch.__version__)
print(onnx.__version__)

In [ ]:
MODEL_PATH = "./result/best.pt" # 모델 경로
ONNX_PATH = "./onnx/panns_cnn14.onnx" # 오닉스 저장경로
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용 가능여부 확인

In [ ]:
print(DEVICE)

In [ ]:
# 모델 로드
model = torch.load(MODEL_PATH, weights_only=False)
model.to(DEVICE)
# print(model.spectrogram_extractor.stft.pad_mode)

In [ ]:
# 더미 생성: (batch_size, length)
dummy_input = torch.randn(1, 1, 101, 64, device=DEVICE)

In [ ]:
print(dummy_input.shape)

In [ ]:
# onnx 모델 변환
print("ONNX 변환중...")

torch.onnx.export(model, # PyTorch 모델
                dummy_input, # 입력 텐서 (튜플도 가능)
                ONNX_PATH, # 저장할 파일 경로
                export_params=True, # 모델 파라미터 포함
                opset_version=11, # onnx의 연산자 버전
                input_names=['input'], # 입력 이름
                output_names=['output'], # 출력 이름
                dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 배치 크기를 동적으로 지정
)

print(f"ONNX 변환완료'{ONNX_PATH}'에 저장됨")

In [ ]:
model = onnx.load(ONNX_PATH)
print(onnx.helper.printable_graph(model.graph))

In [ ]:
print("Inputs:")
for input_tensor in model.graph.input:
    print(input_tensor)

print("\nOutputs:")
for output_tensor in model.graph.output:
    print(output_tensor)

In [ ]:
try:
    onnx.checker.check_model(model)
    print("모델이 올바른 ONNX 형식을 따릅니다.")
except onnx.checker.ValidationError as e:
    print("모델 검증 실패")
    print(e)

In [ ]:
# # 모델 시각화
# netron.start (ONNX_PATH)